In [1]:
import numpy as np
import matplotlib.pylab as plt
import os
import csv
import cPickle
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.platform import flags
from sklearn.cross_validation import train_test_split

/lclhome/dli/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
Dre_dir = './dataset'
save_dir = Dre_dir
save_name = os.path.join(save_dir, 'words_vocab.pkl')
#load 
with open(save_name, 'r') as fload:
    words, vocabs = cPickle.load(fload)
    print('=========words usage=========')
    for i in range(5):
        print(" [%d]: words[%d] is [%s]" % (i, i, words[i]))
    print('=========vocab usage=========')
    for i in range(5):
        print(" [%d]: [vocab[%s]] is [%s]" % (i, words[i], vocabs[words[i]]))

=========words usage=========
 [0]: words[0] is [feature::android.hardware.touchscreen]
 [1]: words[1] is [intent::android.intent.action.MAIN]
 [2]: words[2] is [intent::android.intent.category.LAUNCHER]
 [3]: words[3] is [permission::android.permission.INTERNET]
 [4]: words[4] is [permission::android.permission.ACCESS_NETWORK_STATE]
=========vocab usage=========
 [0]: [vocab[feature::android.hardware.touchscreen]] is [0]
 [1]: [vocab[intent::android.intent.action.MAIN]] is [1]
 [2]: [vocab[intent::android.intent.category.LAUNCHER]] is [2]
 [3]: [vocab[permission::android.permission.INTERNET]] is [3]
 [4]: [vocab[permission::android.permission.ACCESS_NETWORK_STATE]] is [4]


In [3]:
#load data
mal_data_path = os.path.join(save_dir, 'malware.data')
with open(mal_data_path, 'rb') as f_r:
    malware_feature_vectors = np.load(f_r)

ben_data_path = os.path.join(save_dir, 'benign.data') 
with open(ben_data_path, 'rb') as f_r:
    benign_feature_vectors = np.load(f_r)


MemoryError: 

In [4]:
print("malware data shape is %s" % str(malware_feature_vectors.shape))
print("benign data shape is %s" % str(benign_feature_vectors.shape))

malware data shape is (5560, 13641)
benign data shape is (123453, 13641)


In [5]:
#transform static features to num
features_pool = np.vstack([malware_feature_vectors,benign_feature_vectors])

label_binary_onehot = np.zeros((len(features_pool), 2), dtype = np.float32)
label_binary_onehot[:len(malware_feature_vectors), 0] = 1.
label_binary_onehot[len(malware_feature_vectors):, 1] = 1.
label_binary_logist = np.zeros((len(features_pool)))
label_binary_logist[:len(malware_feature_vectors)] = 1.
print(features_pool.shape)

(129013, 13641)


In [6]:
mal_ratio = [0.3, 0.4, 0.2] #the best ratio of malware sample in whole training data is 0.3 
data_num = int(malware_feature_vectors.shape[0] / mal_ratio[0])
data_binary = features_pool[:data_num]
label_onehot = label_binary_onehot[:data_num]
#shuffle and split data
train_percentage = 0.9
test_percentage = 0.1 #Here, we use test dataset as validation dataset
trainX, testX, trainy, testy = train_test_split(data_binary, label_onehot, test_size = test_percentage, random_state = 0)

In [7]:
#check
print(trainX.shape)
np.sum(np.argmax(testy, axis = 1) == 0)

(16679, 13641)


557

In [3]:
def MLP(_x, hidden_units = [128, 128], output_dim = 2, keep_probs = 0.6, reuse = False):
    batch_norm_params = {'is_training': is_training, 'decay': 0.9
                         , 'updates_collections': None}
    fc1 = slim.fully_connected(_x, hidden_units[0], 
                               activation_fn = tf.nn.relu,
                               weights_initializer = initializer,
                               normalizer_fn = slim.batch_norm,
                               normalizer_params = batch_norm_params,
                               scope = 'fc1',
                               reuse = reuse
                              )
    fc2 = slim.fully_connected(fc1, hidden_units[1],
                               activation_fn = tf.nn.relu,
                               weights_initializer = initializer,
                               normalizer_fn = slim.batch_norm,
                               normalizer_params = batch_norm_params,
                               scope = 'fc2',
                               reuse = reuse
                              )
    dropout2 = slim.dropout(fc2, keep_prob = keep_probs, is_training = is_training, scope = 'dropout2')
    fc3 = slim.fully_connected(dropout2, output_dim,
                               activation_fn = None,
                               scope = 'fc3',
                               reuse = reuse
                              )
    return fc3

In [4]:
input_dims = len(vocabs)
output_dims =2
FLAGS = flags.FLAGS
model_dir = './models/'
class _ArgsWrapper(object):
    def __init__(self, args):
        if not isinstance(args, dict):
            args = vars(args)
        self.args = args
    def __getattr__(self, name):
        val = self.args.get(name)
        if val is None:
            MSG = 'Setting params ({}) is deprecated'
            warnings.warn(MSG.format(name))
            val = FLAGS.__getattr__(name)
        return val
    
params = {
    'is_bin_dataset':True,
    'n_epoches':30,
    'batchsize':128,
    'learning_rate':0.001
}
args = _ArgsWrapper(params)
    
tf.reset_default_graph()
x = tf.placeholder(dtype = tf.float32, shape = [None, input_dims], name = 'X')
y = tf.placeholder(dtype = tf.float32, shape = [None, output_dims], name = 'Y')
is_training = tf.placeholder(tf.bool)

initializer = tf.random_normal_initializer(0, 0.01)
mlp = MLP(x, output_dim= output_dims)
mlp_softmax = tf.nn.softmax(mlp)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y, logits= mlp))
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(mlp, axis = 1), tf.argmax(y, axis = 1)), tf.float32))
optimizer = tf.train.AdamOptimizer(args.learning_rate).minimize(loss)
saver = tf.train.Saver()
model_save_dir = "./save/model_tmp"
if not os.path.exists(model_save_dir):
    os.mkdir(model_save_dir)
tf.set_random_seed(1234)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [28]:

with sess.as_default():
    
    best_acc = 0.

    n_batches = trainX.shape[0] // args.batchsize
    iterations = 0.
    display = 100
    for epoch in range(args.n_epoches):
        for mini_index in range(n_batches):
            start_i = mini_index * args.batchsize
            end_i = start_i + args.batchsize
            if end_i > trainX.shape[0]:
                end_i = trainX.shape[0]
            _feed_dicts = {
                x: trainX[start_i: end_i],
                y: trainy[start_i: end_i],
                is_training: True
            }
            _, _loss = sess.run([optimizer, loss], feed_dict = _feed_dicts)
            iterations = mini_index + 1 + epoch * n_batches
            if iterations % display == 0:
                _feed_dicts = {
                    x: testX,
                    y: testy,
                    is_training: False
                }
                _acc = sess.run(acc, feed_dict = _feed_dicts)
                if _acc > best_acc:
                    best_acc = _acc
                    model_save_path = os.path.join(model_save_dir, "model.ckpt")
                    saver.save(sess, model_save_path)
                MSG = "iterations:{0}, training loss: {1:.4}, validation accuracy:{2:.4}, best accuracy:{3:.4}"
                print MSG.format(iterations, _loss, _acc, best_acc)


iterations:100, training loss: 0.0569, validation accuracy:0.9628, best accuracy:0.9628
iterations:200, training loss: 0.1283, validation accuracy:0.9649, best accuracy:0.9649
iterations:300, training loss: 0.03193, validation accuracy:0.9714, best accuracy:0.9714
iterations:400, training loss: 0.02523, validation accuracy:0.9714, best accuracy:0.9714
iterations:500, training loss: 0.01954, validation accuracy:0.9698, best accuracy:0.9714
iterations:600, training loss: 0.02751, validation accuracy:0.966, best accuracy:0.9714
iterations:700, training loss: 0.006414, validation accuracy:0.9698, best accuracy:0.9714
iterations:800, training loss: 0.01093, validation accuracy:0.9687, best accuracy:0.9714
iterations:900, training loss: 0.02914, validation accuracy:0.9676, best accuracy:0.9714
iterations:1000, training loss: 0.009658, validation accuracy:0.9655, best accuracy:0.9714
iterations:1100, training loss: 0.005424, validation accuracy:0.9693, best accuracy:0.9714
iterations:1200, tr

In [5]:
model_save_path = os.path.join(model_save_dir, "model.ckpt")
def evaluate_model(sess, dataX, labely):
    eval_batches = dataX.shape[0] // args.batchsize
    accs = []
    with sess.as_default():
        saver.restore(sess, model_save_path)
        for mini_index in range(eval_batches):
            start_i = mini_index * args.batchsize
            end_i = start_i + args.batchsize
            if end_i > dataX.shape[0]:
                end_i = dataX.shape[0]
            _feed_dicts = {
                x: dataX[start_i: end_i],
                y: labely[start_i: end_i],
                is_training: False
            }
            _acc = sess.run(acc, feed_dict = _feed_dicts)
            accs.append(_acc)
        return np.mean(np.array(accs))

In [30]:
#accuracy 
accuracy = evaluate_model(sess, features_pool, label_binary_onehot)

print("The accuracy of dataset is %.5f" % accuracy)

INFO:tensorflow:Restoring parameters from ./save/model_tmp/model.ckpt
The accuracy of dataset is 0.98763


In [31]:
#False negative rate
mal_num = malware_feature_vectors.shape[0]
FNR = 1. - evaluate_model(sess, features_pool[:mal_num], label_binary_onehot[:mal_num])
#False positive rate
FPR = 1. - evaluate_model(sess, features_pool[mal_num:], label_binary_onehot[mal_num:])
print("False negative rate: %.5f, False positive rate: %.5f" % (FNR, FPR))

INFO:tensorflow:Restoring parameters from ./save/model_tmp/model.ckpt
INFO:tensorflow:Restoring parameters from ./save/model_tmp/model.ckpt
False negative rate: 0.03979, False positive rate: 0.01114


# ================Generating Adversarial Examples================

In [32]:
print("saving training dataset")
train_data_save_path = os.path.join(Dre_dir, "./train.data")
train_label_save_path = os.path.join(Dre_dir, "./train.label")
with open(train_data_save_path, 'wb') as s_f:
    np.save(s_f, trainX)
with open(train_label_save_path, 'wb') as s_f:
    np.save(s_f, trainy)

saving training dataset


In [33]:
print("saving test dataset")
test_data_save_path = os.path.join(Dre_dir, "./test.data")
test_label_save_path = os.path.join(Dre_dir, "./test.label")
with open(test_data_save_path, 'wb') as s_f:
    np.save(s_f, testX)
with open(test_label_save_path, 'wb') as s_f:
    np.save(s_f, testy)
    

saving test dataset


In [6]:
test_data_save_path = os.path.join(Dre_dir, "./test.data")
test_label_save_path = os.path.join(Dre_dir, "./test.label")
with open(test_data_save_path, 'rb') as s_r:
    testX = np.load(s_r)
with open(test_label_save_path, 'rb') as s_r:
    testy = np.load(s_r) 

In [7]:
mal_idx = (testy[:,0] == 1)
malware_of_testX = testX[mal_idx]
malware_of_testy = testy[mal_idx]
print(malware_of_testy[:5])

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


In [8]:
accuracy = evaluate_model(sess, malware_of_testX, malware_of_testy)

print accuracy


INFO:tensorflow:Restoring parameters from ./save/model_tmp/model.ckpt
0.935547


In [9]:
with sess.as_default():
    _feed_dicts = {
        x: malware_of_testX,
        y: malware_of_testy,
        is_training: False
    }
    _softmax = sess.run(mlp_softmax, feed_dict = _feed_dicts)
correct_idx = np.argmax(_softmax, axis = 1) == np.argmax(malware_of_testy, axis = 1)
print(malware_of_testX.shape)
malware_of_testX = malware_of_testX[correct_idx]
malware_of_testy = malware_of_testy[correct_idx]
print(malware_of_testX.shape)

(557, 13641)
(519, 13641)


# 1.FGSM

In [14]:
#Fast gradient sign method
def fgsm_graph(x, preds): #[batch_size, vector_dim]
    #avoid label leaking
    preds_max = tf.reduce_max(preds, axis = 1, keep_dims = True)
    preds_y = tf.to_float(tf.equal(preds_max, preds))
    preds_y = tf.stop_gradient(preds_y)
    preds_y = preds_y / tf.reduce_sum(preds_y, 1, keep_dims= True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= preds_y, logits= preds))
    grad, = tf.gradients(loss, x)
    return tf.sign(grad), grad

In [15]:
def fgsm(sess, X, label, invalid_positions, top_k = 20, metrics = "L0"):
    grads_sign, grads = fgsm_graph(x, mlp)
    with sess.as_default():
        _feed_dict = {
            x:X,
            y:label,
            is_training: False
        }
        _grads_sign, _grads = sess.run([grads_sign, grads], feed_dict = _feed_dict)
    #_grad_sign = np.reshape(_grad_sign[0], _grad_sign.shape[-2:])
    _grads[invalid_positions] = np.min(_grads)
    top_k_batch = np.argpartition(_grads, np.argmin(_grads, axis = 0))[:, -top_k:]
    adv_X = np.copy(X)
    for i, mal in enumerate(adv_X):
        mal[top_k_batch[i]] = 1.
    return np.clip(adv_X, a_min = 0, a_max = 1.)

In [16]:
adv_smp_fgsm = []
adv_batches = malware_of_testX.shape[0] // args.batchsize + 1
with sess.as_default():
    for mini_b in range(adv_batches):
        start_i = mini_b * args.batchsize
        end_i = start_i + args.batchsize
        if end_i > malware_of_testX.shape[0]:
            end_i = malware_of_testX.shape[0]
        mal_data = malware_of_testX[start_i : end_i]
        mal_label = malware_of_testy[start_i : end_i]
        invalid_pos = (mal_data == 1.)
        adv_smp_fgsm.append(fgsm(sess, mal_data, mal_label, invalid_pos, top_k= 50))


In [14]:
def evaluate_samples(sess, samples, labels):
    with sess.as_default():
        _feed_dict = {
            x:samples,
            y:labels,
            is_training:False
        }
        return sess.run(acc, feed_dict = _feed_dict)

In [18]:
print evaluate_samples(sess, malware_of_testX, malware_of_testy)
print evaluate_samples(sess, np.squeeze(np.array(np.vstack(adv_smp_fgsm))), malware_of_testy)

1.0
0.504817


In [19]:
#experiments
perturbations = [10, 20, 30, 50, 80, 200]
def get_adv_examples(perturbs):
    adv_smp_fgsm = []
    adv_batches = malware_of_testX.shape[0] // args.batchsize + 1
    with sess.as_default():
        for mini_b in range(adv_batches):
            start_i = mini_b * args.batchsize
            end_i = start_i + args.batchsize
            if end_i > malware_of_testX.shape[0]:
                end_i = malware_of_testX.shape[0]
            mal_data = malware_of_testX[start_i : end_i]
            mal_label = malware_of_testy[start_i : end_i]
            invalid_pos = (mal_data == 1.)
            adv_smp_fgsm.append(fgsm(sess, mal_data, mal_label, invalid_pos, top_k= perturbs))
    return np.array(np.vstack(adv_smp_fgsm))

In [20]:
adv_smp_10 = get_adv_examples(perturbations[0])
adv_smp_20 = get_adv_examples(perturbations[1])
adv_smp_30 = get_adv_examples(perturbations[2])
adv_smp_50 = get_adv_examples(perturbations[3])
adv_smp_80 = get_adv_examples(perturbations[4])
adv_smp_200 = get_adv_examples(perturbations[5])

In [21]:
print evaluate_samples(sess, malware_of_testX, malware_of_testy)
print evaluate_samples(sess, adv_smp_10, malware_of_testy)
print evaluate_samples(sess, adv_smp_20, malware_of_testy)
print evaluate_samples(sess, adv_smp_30, malware_of_testy)
print evaluate_samples(sess, adv_smp_50, malware_of_testy)
print evaluate_samples(sess, adv_smp_80, malware_of_testy)
print evaluate_samples(sess, adv_smp_200, malware_of_testy)

1.0
0.732177
0.583815
0.533719
0.504817
0.435453
0.306358


In [ ]:
import utils
adv_smps_dir = './dataset/adv_smps/ML/'
dnn_advsmps_name = 'dnn_adv_smp.data'
dnn_advsmps_label = 'dnn_adv_smp.label'
ldh_advsmps_name = 'ldh_adv_smp10.data'
ldh_advsmps_label = 'ldh_adv_smp10.label'

dt_advsmps_name = 'dt_adv_smp10.data.npy'
dt_advsmps_label = 'dt_adv_smp10.label.npy'
lr_advsmps_name = 'lr_adv_smp10.data'
lr_advsmps_label = 'lr_adv_smp10.label'
svm_advsmps_name = 'svm_adv_smp10.data.npy'
svm_advsmps_label = 'svm_adv_smp10.label.npy'
clean_data_name = 'clean_smp.data'
clean_label_label = 'clean_smp.label'
clean_data_lr_name = 'lr_clean.data.npy'
clean_label_lr_label = 'lr_clean.label.npy'
clean_data_ldh_name = 'ldh_clean.data.npy'
clean_label_ldh_label = 'ldh_clean.label.npy'

DNN_advsmps_data_path = os.path.join(adv_smps_dir, dnn_advsmps_name)
DNN_advsmps_label_path = os.path.join(adv_smps_dir, dnn_advsmps_label)
LDH_advsmps_data_path = os.path.join(adv_smps_dir, ldh_advsmps_name)
LDH_advsmps_label_path = os.path.join(adv_smps_dir, ldh_advsmps_label)
DT_advsmps_data_path = os.path.join(adv_smps_dir, dt_advsmps_name)
DT_advsmps_label_path = os.path.join(adv_smps_dir, dt_advsmps_label)
LR_advsmps_data_path = os.path.join(adv_smps_dir, lr_advsmps_name)
LR_advsmps_label_path = os.path.join(adv_smps_dir, lr_advsmps_label)
SVM_advsmps_data_path = os.path.join(adv_smps_dir, svm_advsmps_name)
SVM_advsmps_label_path = os.path.join(adv_smps_dir, svm_advsmps_label)
clean_data_path = os.path.join(adv_smps_dir, clean_data_name)
clean_label_path = os.path.join(adv_smps_dir, clean_label_label)
clean_data_lr_path = os.path.join(adv_smps_dir, clean_data_lr_name)
clean_label_lr_path = os.path.join(adv_smps_dir, clean_label_lr_label)
clean_data_ldh_path = os.path.join(adv_smps_dir, clean_data_ldh_name)
clean_label_ldh_path = os.path.join(adv_smps_dir, clean_label_ldh_label)

DNN_advsmps_data = utils.readdata_np(DNN_advsmps_data_path)
DNN_advsmps_label = utils.readdata_np(DNN_advsmps_label_path)
LDH_advsmps_data = utils.readdata_np(LDH_advsmps_data_path)
LDH_advsmps_label = utils.readdata_np(LDH_advsmps_label_path)
DT_advsmps_data = utils.readdata_np(DT_advsmps_data_path)
DT_advsmps_label = utils.readdata_np(DT_advsmps_label_path)
LR_advsmps_data = utils.readdata_np(LR_advsmps_data_path)
LR_advsmps_label = utils.readdata_np(LR_advsmps_label_path)
SVM_advsmps_data = utils.readdata_np(SVM_advsmps_data_path)
SVM_advsmps_label = utils.readdata_np(SVM_advsmps_label_path)
clean_data = utils.readdata_np(clean_data_path)
clean_label = utils.readdata_np(clean_label_path)
clean_data_lr = utils.readdata_np(clean_data_lr_path)
clean_label_lr = utils.readdata_np(clean_label_lr_path)
clean_data_ldh = utils.readdata_np(clean_data_ldh_path)
clean_label_ldh = utils.readdata_np(clean_label_ldh_path)

In [ ]:
acc_on_cl = evaluate_samples(sess, clean_data, clean_label) 
acc_on_lr_cl = evaluate_samples(sess, clean_data_lr, clean_label_lr) 
acc_on_ldh_cl = evaluate_samples(sess, clean_data_ldh, clean_label_ldh)
print acc_on_cl, acc_on_lr_cl, acc_on_ldh_cl
print evaluate_samples(sess, DNN_advsmps_data, DNN_advsmps_label)
print 1. - evaluate_samples(sess, LDH_advsmps_data, LDH_advsmps_label)
dt_onehot = np.zeros((len(DT_advsmps_label), 2)).astype(np.float32)
dt_onehot[:,0] = 1.
print 1. - evaluate_samples(sess, DT_advsmps_data, dt_onehot)
print 1. - evaluate_samples(sess, LR_advsmps_data, LR_advsmps_label)
svm_onehot = np.zeros((len(SVM_advsmps_label), 2)).astype(np.float32)
svm_onehot[:,0] = 1.
print 1. - evaluate_samples(sess, SVM_advsmps_data, svm_onehot)

In [22]:
#check
print(adv_smp_10[:5])
print np.sum(adv_smp_10 - malware_of_testX) / malware_of_testX.shape[0]

[[ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]]
10.0


In [23]:
#save 
save_dir = os.path.join(Dre_dir, 'adv_smps/')
save_path1 = os.path.join(save_dir, 'adv_smp10.data')
save_path2 = os.path.join(save_dir, 'adv_smp20.data')
save_path3 = os.path.join(save_dir, 'adv_smp30.data')
save_path4 = os.path.join(save_dir, 'adv_smp50.data')
save_path5 = os.path.join(save_dir, 'adv_smp80.data')
save_path6 = os.path.join(save_dir, 'adv_smp200.data')
save_path7 = os.path.join(save_dir, 'clean_smp.data')
save_path8 = os.path.join(save_dir, 'clean_smp.label')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
with open(save_path1, 'wb') as fsave:
    np.save(fsave, adv_smp_10)
    print("the adversarial data is saved to [%s]" % save_path1)
    
with open(save_path2, 'wb') as fsave:
    np.save(fsave, adv_smp_20)
    print("the adversarial data is saved to [%s]" % save_path2)
    
with open(save_path3, 'wb') as fsave:
    np.save(fsave, adv_smp_30)
    print("the adversarial data is saved to [%s]" % save_path3)
    
with open(save_path4, 'wb') as fsave:
    np.save(fsave, adv_smp_50)
    print("the adversarial data is saved to [%s]" % save_path4)
    
with open(save_path5, 'wb') as fsave:
    np.save(fsave, adv_smp_80)
    print("the adversarial data is saved to [%s]" % save_path5)

with open(save_path6, 'wb') as fsave:
    np.save(fsave, adv_smp_200)
    print("the adversarial data is saved to [%s]" % save_path6)
    
with open(save_path7, 'wb') as fsave:
    np.save(fsave, malware_of_testX)
    print("the adversarial data is saved to [%s]" % save_path7)
    
with open(save_path8, 'wb') as fsave:
    np.save(fsave, malware_of_testy)
    print("the adversarial data is saved to [%s]" % save_path8)    

the adversarial data is saved to [./dataset/adv_smps/adv_smp10.data]
the adversarial data is saved to [./dataset/adv_smps/adv_smp20.data]
the adversarial data is saved to [./dataset/adv_smps/adv_smp30.data]
the adversarial data is saved to [./dataset/adv_smps/adv_smp50.data]
the adversarial data is saved to [./dataset/adv_smps/adv_smp80.data]
the adversarial data is saved to [./dataset/adv_smps/adv_smp200.data]
the adversarial data is saved to [./dataset/adv_smps/clean_smp.data]
the adversarial data is saved to [./dataset/adv_smps/clean_smp.label]


In [33]:
#save 
save_dir = "."
import cPickle
save_path2 = os.path.join(save_dir, 'binary_seqs_modify100_Android.data')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

    
with open(save_path2, 'wb') as fsave:
    cPickle.dump(np.squeeze(np.array(dist_smp_fgsm)), fsave)
    print("the distortion data is saved to [%s]" % save_path2)

with open(save_path2, 'rb') as fload:
    dist_feat = cPickle.load(fload)
    print("====================the save data=================")
    print dist_feat[0, :]

the distortion data is saved to [./binary_seqs_modify100_Android.data]
====================the save data=================
[ 1.  1.  1. ...,  0.  0.  0.]


# 2. Optimization-based approach 

In [10]:
#inference: x' = x + dist * valid
batch_size = 1
valid_M = tf.placeholder(dtype = tf.float32, shape = [None, input_dims], name = "VALID_M")
dist_M = tf.Variable(tf.zeros([batch_size, input_dims]))
dist_feat = x + dist_M * valid_M
dist_mlp_out = MLP(dist_feat, output_dim= output_dims, reuse = True)
#loss1 = tf.log(tf.constant(2.))-tf.nn.softplus(tf.reduce_sum(dist_mlp_out * y, axis = 1) - tf.reduce_max(dist_mlp_out * (1. - y), axis = 1))
#loss1 = -tf.maximum(tf.reduce_sum(dist_mlp_out * y, axis = 1) - tf.reduce_max(dist_mlp_out * (1. - y), axis = 1), 0.0)
#loss1 = -tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= dist_mlp_out, labels= 1. -y))
loss1 = tf.log(1. - tf.reduce_mean(tf.nn.softmax(dist_mlp_out) * y ,axis = 1) + 1e-6)
loss2 = tf.reduce_sum(tf.square(dist_M * valid_M))
loss = loss1 + loss2
optimizer_dis = tf.train.AdamOptimizer()
trainer = optimizer_dis.minimize(loss, var_list = [dist_M])
guide_grad = tf.gradients(loss1, dist_feat)

In [11]:
is_vars_inited_graph = [tf.is_variable_initialized(var) for var in tf.global_variables()]
#tf.set_random_seed(1234)
def logist_zero_sum_approach(sess, feat, label, valid, maxIters = 100, top_k = 2):    
    with sess.as_default():
        is_inited = sess.run(is_vars_inited_graph)
        uninited_vars = [var for (var, init) in 
                         zip(tf.global_variables(), is_inited) if not init]
        if len(uninited_vars):
            sess.run(tf.variables_initializer(uninited_vars))

        _feed_dict = {
                x: feat,
                y: label,
                valid_M: valid,
                is_training:False
            }
        display_ = int(maxIters * 0.1)
        argminset = set()
        for idx in range(maxIters):
            _, _loss_dist, _loss1, _grads, _dist_feat= \
            sess.run([trainer, loss, loss1,  guide_grad, dist_feat], feed_dict = _feed_dict)
           
            if idx % display_ == 0:
                #check
                _grads_valid = _grads * valid
                _grads_flat = np.reshape(_grads_valid, -1)
                _pre_invalid_count = int(np.sum(valid) * 0.1)
                argmin_sort = np.argsort(_grads_flat)
                argmin = []
                for _i, _e in enumerate(argmin_sort[:-top_k]):
                    if _e not in argminset:
                        argminset.add(_e)
                        argmin.append(_e)
                    if len(argmin) >= _pre_invalid_count:
                        break
                
                argmax = np.argpartition(_grads_flat, -top_k )[-top_k:]
                valid[:, argmin] = 0.
                _feed_dict = {x:feat, y:label, valid_M: valid, is_training: False}
            
    return argmax, _grads, valid, _dist_feat

In [16]:
'''
dist_smp_fgsm = []
valid_v = np.array([vocabs_modify[key] for key in words])
valid_v_bool = valid_v.astype(np.bool)
for _attack_smp in pre_attack_samples:
    valid_ = _attack_smp == 0.
    invalid = 1 - (valid_v_bool + valid_)
    _attack_smp = np.reshape(_attack_smp, (1, _attack_smp.shape[0]))
    _label = np.array([[1, 0]], dtype = np.float32)
    _dist_smp = fgsm(sess, _attack_smp, _label, invalid, max_dist_ratio = 0.0001)
    dist_smp_fgsm.append(_dist_smp)
'''

select_num = 1
seqs= []
seqs_modify = []
for i, malware in enumerate(malware_of_testX):
    seq = malware.reshape((1, -1)).copy()
    seq_backup = seq.copy()
    label = np.array([[1., 0.]], dtype = np.float64)
    #valid_i = np.clip(seq, a_min = 0, a_max = 1).astype(np.int).reshape((1, -1))
    valid_i = 1. - seq
    valid_count = np.sum(valid_i)
    for distort_idx in range(10):
        argmax, grad_status, update_valid, modified_seq = \
        logist_zero_sum_approach(sess, seq, label, valid_i, top_k = select_num)
        print "[%d]:The count of valid features is %d/%d" % (i, np.sum(update_valid), valid_count)
        if np.sum(update_valid) == 0:
            print "Warning : No valid featrue can be perturbed"
        #perturbation
        seq[:, argmax] = 1. 
        print "The unit of pertubation is %s" % argmax
        #check
        checked_acc = evaluate_samples(sess, seq, label)
        if checked_acc == 0:
            print "Success!"
        #update
        valid_i[:, argmax] = 0.
        #store
    seqs.append(seq_backup)
    seqs_modify.append(seq)
    print("-----------------------------------------------------------")

[0]:The count of valid features is 4766/13622
The unit of pertubation is [13238]
[0]:The count of valid features is 4735/13622
The unit of pertubation is [4827]
[0]:The count of valid features is 4662/13622
The unit of pertubation is [1734]
[0]:The count of valid features is 4421/13622
The unit of pertubation is [10197]
[0]:The count of valid features is 4219/13622
The unit of pertubation is [5351]
[0]:The count of valid features is 4050/13622
The unit of pertubation is [1726]
Success!
[0]:The count of valid features is 3979/13622
The unit of pertubation is [184]
Success!
[0]:The count of valid features is 3931/13622
The unit of pertubation is [1682]
Success!
[0]:The count of valid features is 3874/13622
The unit of pertubation is [11701]
Success!
[0]:The count of valid features is 3828/13622
The unit of pertubation is [2530]
Success!
-----------------------------------------------------------
[1]:The count of valid features is 4763/13631
The unit of pertubation is [4433]
[1]:The count

In [17]:
print evaluate_samples(sess, np.squeeze(seqs_modify), malware_of_testy)

0.00385356


In [18]:
print np.mean(np.sum(np.abs(np.squeeze(np.array(seqs_modify)) - malware_of_testX),axis = 1))

10.0


In [24]:
#save 
save_dir = "./dataset/"
import cPickle
save_path_data = os.path.join(save_dir, 'adv_smps_opt.data')
save_path_label = os.path.join(save_dir, 'adv_smps_opt.label')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
with open(save_path_data, 'wb') as f:
    np.save(f, np.squeeze(np.array(seqs_modify)))
    
with open(save_path_label, 'wb') as f:
    np.save(f, malware_of_testy)

dist_feat_load = np.load(save_path)


print("====================the save data=================")
print dist_feat_load[0, :]

====================the save data=================
[ 1.  1.  1. ...,  0.  0.  0.]
